<h1 style="text-align:center;">Travailler Avec des Tableaux dans R</h1>


<u>**Auteur</u> :** [Younes Dahami](https://www.linkedin.com/in/dahami/)

# Introduction

Les tableaux sont souvent essentielles pour organiser et résumer nos données, notamment avec des variables catégorielles. Lorsque l'on crée un tableau dans R, elle considère notre tableau comme un type d'objet spécifique (appelé "table") qui est très similaire à un dataframe. Bien que cela puisse sembler étrange puisque les datasets sont stockés sous forme de dataframes. Dans ce notebook, nous discuterons de la création de divers types de tables et de l'utilisation de différentes méthodes statistiques pour analyser les données tabulaires. Tout au long du notebook, la dataset AOSI sera utilisé.

# 1) Création de tables de base : `table()` , `xtabs()` et `ltabs()`

## 1.1) `table()`

Un tableau de contingence ("tri à plat" ou "Tableau de fréquences) est une tabulation des comptes et/ou pour une ou plusieurs variables. En R, ces tables peuvent être créées à l'aide de **`table()`** ainsi que certaines de ses variations.

Pour utiliser `table()`, on ajoute simplement les variables qu'on souhaite tabuler séparées par une virgule. Remarquons que `table()` n'a pas d'argument data= comme de nombreuses autres fonctions le font (par exemple, les fonctions ggplot2), donc nous devons référencer la variable en utilisant **`dataset$variable`**. Quelques exemples sont présentés ci-dessous. Par défaut, les valeurs manquantes sont exclues des décomptes, si nous souhaitons un décompte pour ces valeurs manquantes, nous devons spécifier l'argument **`useNA="ifany"`** ou **`useNA="always"`**. Les exemples ci-dessous montrent comment utiliser cette fonction.

* **`useNA = "no"` :** (valeur par défaut), les valeurs manquantes ne sont jamais incluses dans le tri à plat.

* **`useNA = "ifany` :** (if any), une colonne NA est ajoutée si des valeurs manquantes sont présentées dans les données.

* **`useNA = "always"`** une colonne NA est toujours ajoutée, même s'il n'ya pas de valeurs manquantes dans les données.


In [2]:
data_aosi <- read.csv("cross-sec_aosi.csv", stringsAsFactors = FALSE, na.strings = ".")

In [4]:
# Table par sexe
table(data_aosi$Gender)


Female   Male 
   235    352 

In [5]:
# Table par Site d'étude
table(data_aosi$Study_Site)


PHI SEA STL UNC 
149 152 145 141 

Quand le nombre de modalité est élevé, on peut ordonner le tri à plat selon les effectifs à l'aide de la fonction `sort()` :

In [9]:
sort(table(data_aosi$Study_Site))


UNC STL PHI SEA 
141 145 149 152 

In [11]:
sort(table(data_aosi$Study_Site), decreasing = TRUE)


SEA PHI STL UNC 
152 149 145 141 

In [7]:
# table à deux voix pour sexe et site d'étude (two-way table)
table(data_aosi$Gender, data_aosi$Study_Site)

        
         PHI SEA STL UNC
  Female  55  67  60  53
  Male    94  85  85  88

Remarquons que l'ordre est important : la 1ère variable est la variable de ligne, la 2ème variable est la variable de colonne. Essayons d'ajouter l'argument `useNA` :

In [8]:
table(data_aosi$Gender, data_aosi$Study_Site, useNA = "ifany")

        
         PHI SEA STL UNC
  Female  55  67  60  53
  Male    94  85  85  88

In [13]:
table(data_aosi$Gender, data_aosi$Study_Site, useNA = "always")

        
         PHI SEA STL UNC <NA>
  Female  55  67  60  53    0
  Male    94  85  85  88    0
  <NA>     0   0   0   0    0

In [14]:
# Sauvons l'une de ces tables pour une utilisation ultérieure dans des exemples
table_exp <- table(data_aosi$Gender, data_aosi$Study_Site)


Ajoutons des étiquettes de lignes et de colonnes au tableau `Gender` par site d'étude. Pour un objet de tableau, ces étiquettes sont appelées **dimnames** (c'est-à-dire noms de dimensions) qui peuvent être accédées en utilisant la fonction **`dimnames()`**. 

Notons que c'est similaire à la fonction **`names()`** avec les listes, sauf que maintenant notre tableau a plusieurs dimensions, chacune pouvant avoir son propre ensemble de noms.

**Pour un tableau, les dimnames sont stockés sous forme de liste, chaque entrée de liste contenant les étiquettes de groupe pour la variable correspondante à cette dimension.** Le nom de chacune de ces entrées de liste spécifie l'étiquette réelle à utiliser dans le tableau. Par défaut, ces noms sont vides, c'est pourquoi le tableau par défaut n'a pas d'étiquettes de lignes et de colonnes. Nous pouvons changer cela en spécifiant ces noms, en utilisant **`names()`** avec **`dimnames()`**.


In [15]:
dimnames(table_exp)

[[1]]
[1] "Female" "Male"  

[[2]]
[1] "PHI" "SEA" "STL" "UNC"

On voit les étiquettes de groupe. Notons que chaque ensemble d'étiquettes de groupe est non nommé (vides à côté de [[1]] et [[2]]). Cela est plus clairement visible en accédant explicitement à ces noms en utilisant `names()`


In [16]:
names(dimnames(table_exp))

[1] "" ""

In [18]:
# changeons les noms

names(dimnames(table_exp)) <- c("Gender", "Site")
names(dimnames(table_exp))

[1] "Gender" "Site"

In [19]:
table_exp

        Site
Gender   PHI SEA STL UNC
  Female  55  67  60  53
  Male    94  85  85  88

Maintenant, les étiquettes des lignes et des colonnes apparaissent, ce qui rend le tableau plus facile à comprendre.

Il est également courant de visualiser ces tabulations sous forme de **pourcentages.** Cela peut être fait en utilisant **`prop.table()`**, qui contrairement à `table()`, **prend comme argument un objet table** et non les variables d'intérêt réelles.

Notons que toutes les modifications apportées aux noms de dimensions de l'objet table sont conservées lors de l'application de `prop.table()`. La sortie de `prop.table()` est également stockée comme un objet de type `table` :


In [21]:
# Tableau croisé (2-way table) de pourcentage
prop_table_exp <- prop.table(table_exp)
prop_table_exp

        Site
Gender          PHI        SEA        STL        UNC
  Female 0.09369676 0.11413969 0.10221465 0.09028961
  Male   0.16013629 0.14480409 0.14480409 0.14991482

L'utilisation de **`summary()`** permet également l'affichage du tri à plat et du nombre de non-réponse :

In [24]:
summary(data_aosi$Gender)

   Length     Class      Mode 
      587 character character 

Pour obtenir un tableau avec la répartition en pourcentages, on peut utiliser la fonction **`freq`** de l’extension **`questionr`** :

In [25]:
library(questionr)

In [26]:
freq(data_aosi$Study_Site)

,n,%,val%
,<dbl>,<dbl>,<dbl>
PHI,149,25.4,25.4
SEA,152,25.9,25.9
STL,145,24.7,24.7
UNC,141,24.0,24.0


La colonne `n` donne les effectifs bruts, la colonne `%` la répartition en pourcentages et `val%` la répartition en pourcentages, données manquantes exclues.

La fonction accepte plusieurs paramètres permettant d’afficher les totaux, les pourcentages cumulés, de trier selon les effectifs ou de contrôler l’affichage. Par exemple :

In [27]:
freq(data_aosi$Study_Site, cum=TRUE, total=TRUE, sor="inc", digits=2, exclude = NA)

,n,%,%cum
,<dbl>,<dbl>,<dbl>
UNC,141,24.02,24.02
STL,145,24.70,48.72
PHI,149,25.38,74.11
SEA,152,25.89,100.00
Total,587,100.00,100.00


La colonne `%cum` indique ici le pourcentage cumulé, ce qui est ici une très mauvaise idée puisque pour ce type de variable cela n’a aucun sens. Les lignes du tableau résultat ont été triés par effectifs croissants, les totaux ont été ajoutés, les non-réponses exclues et les pourcentages arrondis à deux décimales.

La fonction `freq` est également en mesure de tenir compte des étiquettes de valeurs lorsqu’on utilise des données labellisées. Ainsi :

In [31]:
data(fecondite)
questionr::describe(femmes$region)

[2000 obs.] Région de résidence
labelled double: 4 4 4 4 4 3 3 3 3 3 ...
min: 1 - max: 4 - NAs: 0 (0%) - 4 unique values
4 value labels: [1] Nord [2] Est [3] Sud [4] Ouest

             n     %
[1] Nord   707  35.4
[2] Est    324  16.2
[3] Sud    407  20.3
[4] Ouest  562  28.1
Total     2000 100.0

In [32]:
freq(femmes$region)

,n,%,val%
,<dbl>,<dbl>,<dbl>
[1] Nord,707,35.4,35.4
[2] Est,324,16.2,16.2
[3] Sud,407,20.3,20.3
[4] Ouest,562,28.1,28.1


In [33]:
freq(femmes$region, levels = "labels")

,n,%,val%
,<dbl>,<dbl>,<dbl>
Nord,707,35.4,35.4
Est,324,16.2,16.2
Sud,407,20.3,20.3
Ouest,562,28.1,28.1


Pour plus d’informations sur la fonction `freq`, consultez sa page d’aide en ligne avec `?freq` ou `help("freq")`.

## 1.2) `xtabs()`

Un deuxième moyen de créer des tableaux de contingence est d'utiliser la fonction **`xtabs()`**, qui nécessite le package **`stats`** (qui est inclus dans R par défaut, bien qu'on devait toujours charger le package avec **library()**).

La fonction `xtabs()` crée un objet de type `xtabs` et on remarquera que la sortie de `xtabs()` et de `table()` est presque identique. `xtabs()` présente les avantages suivants :

1) les étiquettes de lignes et de colonnes sont incluses automatiquement, définies sur les noms des variables.

2) il existe un argument `data=`, ce qui signifie qu'on n'a qu'à référencer les noms des variables. Avec `xtabs()`, on ne liste pas les variables d'intérêt séparées par des virgules. Au lieu de cela, on utilise la notation de formule, qui est :
**<center>~variable1+variable2+...</center>**

où variable1 et variable2 sont les noms des variables d'intérêt. VOn peut ajouter plus de deux variables (d'où les ...). Voir ci-dessous pour l'exemple à deux variables Gender et site :


In [34]:
library(stats)

In [36]:
table_exp_xtabs <- xtabs(~Gender+Study_Site, data = data_aosi)
table_exp_xtabs

        Study_Site
Gender   PHI SEA STL UNC
  Female  55  67  60  53
  Male    94  85  85  88


Pour créer un tableau de proportions en utilisant `xtab()`, on crée d'abord le tableau des dénombrements en utilisant `xtab()`, puis on utilise la fonction `prop.table()` sur cet objet de tableau. C'est exactement ce qui a été fait lors de l'utilisation de `table()`.

Une fonction utile lors de la création de tables de proportions est **`round()`**. Comme vu avec le tableau de proportions précédent, R n'arrondira pas les décimales par défaut. La fonction `round()` peut être utilisée pour tous les types d'objets R. Le premier argument est l'objet de valeurs que l'on souhaite arrondir et le deuxième argument est le nombre de décimales à arrondir :

In [37]:
prop_table_exp_xtabs <- round(prop.table(table_exp_xtabs), digits = 2)
prop_table_exp_xtabs

        Study_Site
Gender    PHI  SEA  STL  UNC
  Female 0.09 0.11 0.10 0.09
  Male   0.16 0.14 0.14 0.15

In [38]:
prop_table_exp <- round(prop_table_exp, 2)
prop_table_exp

        Site
Gender    PHI  SEA  STL  UNC
  Female 0.09 0.11 0.10 0.09
  Male   0.16 0.14 0.14 0.15

Enfin, nous discutons de la manière d'ajouter des totaux marginaux à notre tableau. Que nous utilisions `table()` ou `xtab()`, un moyen simple d'ajouter tous les totaux marginaux à votre tableau est d'utiliser la fonction **`addmargins()`** du package `stats`.

Il suffit d'ajouter notre objet de `table` ou `xtab` comme premier argument à la fonction `addmargins()`, et un nouveau tableau sera renvoyé qui inclut ces totaux marginaux. Cela fonctionne également avec les tables de proportions.


In [39]:
table_exp <- addmargins(table_exp)
table_exp_xtabs <- addmargins(table_exp_xtabs)
prop_table_exp <- addmargins(prop_table_exp)
prop_table_exp_xtabs <- addmargins(prop_table_exp_xtabs)

In [40]:
table_exp

,PHI,SEA,STL,UNC,Sum
Female,55,67,60,53,235
Male,94,85,85,88,352
Sum,149,152,145,141,587


In [41]:
table_exp_xtabs

,PHI,SEA,STL,UNC,Sum
Female,55,67,60,53,235
Male,94,85,85,88,352
Sum,149,152,145,141,587


In [42]:
prop_table_exp

,PHI,SEA,STL,UNC,Sum
Female,0.09,0.11,0.10,0.09,0.39
Male,0.16,0.14,0.14,0.15,0.59
Sum,0.25,0.25,0.24,0.24,0.98


In [43]:
prop_table_exp_xtabs

,PHI,SEA,STL,UNC,Sum
Female,0.09,0.11,0.10,0.09,0.39
Male,0.16,0.14,0.14,0.15,0.59
Sum,0.25,0.25,0.24,0.24,0.98


In [44]:
dimnames(prop_table_exp_xtabs)

$Gender
[1] "Female" "Male"   "Sum"   

$Study_Site
[1] "PHI" "SEA" "STL" "UNC" "Sum"

## 1.3) `ltabs()`

Si l’on utilise des **données labellisées,** la fonction `xtabs` ne prendra pas en compte les étiquettes de valeur comme montré ci-dessous :

In [45]:
data(fecondite)
questionr::describe(femmes$region)

[2000 obs.] Région de résidence
labelled double: 4 4 4 4 4 3 3 3 3 3 ...
min: 1 - max: 4 - NAs: 0 (0%) - 4 unique values
4 value labels: [1] Nord [2] Est [3] Sud [4] Ouest

             n     %
[1] Nord   707  35.4
[2] Est    324  16.2
[3] Sud    407  20.3
[4] Ouest  562  28.1
Total     2000 100.0

In [46]:
xtabs(data = femmes, formula = ~educ+region)

    region
educ   1   2   3   4
   0 387 213 282 256
   1 179  53  86 142
   2 123  57  37 131
   3  18   1   2  33

On pourra alors utiliser la fonction **`ltabs()`** de l’extension **`question`,** qui fonctionne exactement comme `xtabs`, à ceci près qu’elle prendra en compte les étiquettes de variable et de valeur quand elles existent :

In [48]:
ltabs(~educ+region, femmes)

                        region: Région de résidence
educ: Niveau d'éducation [1] Nord [2] Est [3] Sud [4] Ouest
          [0] aucun           387     213     282       256
          [1] primaire        179      53      86       142
          [2] secondaire      123      57      37       131
          [3] supérieur        18       1       2        33

# 2) Pourcentages en ligne et en colonne

On n’a cependant que les effectifs, ce qui rend difficile les comparaisons. L’extension **`questionr`** fournit des fonctions permettant de calculer facilement les pourcentages lignes, colonnes et totaux d’un tableau croisé.

Les pourcentages lignes s’obtiennent avec la fonction **`lprop()`**, dont l'argument, comme pour la fonction `prop.tab()`, est un tableau.
Celle-ci s’applique au tableau croisé généré par table ou xtabs  :

In [62]:
tab <- table(data_aosi$Gender, data_aosi$Study_Site)
lprop(tab)

        
         PHI   SEA   STL   UNC   Total
  Female  23.4  28.5  25.5  22.6 100.0
  Male    26.7  24.1  24.1  25.0 100.0
  All     25.4  25.9  24.7  24.0 100.0

In [50]:
tab <- xtabs(~Gender+Study_Site, data_aosi)
lprop(tab)

        Study_Site
Gender   PHI   SEA   STL   UNC   Total
  Female  23.4  28.5  25.5  22.6 100.0
  Male    26.7  24.1  24.1  25.0 100.0
  All     25.4  25.9  24.7  24.0 100.0

Les pourcentages ligne ne nous intéressent guère ici. On cherche plutôt quelle est la proportion des sexes dans les sites d'étude. Il nous faut donc des pourcentages colonnes, que l’on obtient avec la fonction **`cprop()`** :

In [51]:
cprop(tab)

        Study_Site
Gender   PHI   SEA   STL   UNC   All  
  Female  36.9  44.1  41.4  37.6  40.0
  Male    63.1  55.9  58.6  62.4  60.0
  Total  100.0 100.0 100.0 100.0 100.0

In [52]:
prop(tab)

        Study_Site
Gender   PHI   SEA   STL   UNC   Total
  Female   9.4  11.4  10.2   9.0  40.0
  Male    16.0  14.5  14.5  15.0  60.0
  Total   25.4  25.9  24.7  24.0 100.0

# 3) Analyse des données tabulaires

Dans cette section, nous détaillons quelques méthodes statistiques courantes utilisées pour analyser les données de tableaux de contingence ainsi que la manière de mettre en œuvre ces méthodes dans R. Ces méthodes sont définies et les statistiques qui les sous-tendent sont expliquées, puis leur mise en œuvre dans R est discutée et illustrée à travers des exemples.

## 3.1) Tests d'indépendance

### 3.1.1) Définition de l'indépendance

Supposons que nous ayons deux variables catégorielles, notées $X$ et $Y$. Dénotez la distribution conjointe de $X$ et $Y$ par $f_{x,y}$, la distribution de $X$ par $f_x$ et la distribution de $Y$ par $f_y$. Dénotez la distribution de $X$ conditionnelle à $Y$ par $f_{x|y}$ et la distribution de $Y$ conditionnelle à $X$ par $f_{y|x}$.

En statistiques, $X$ et $Y$ sont indépendantes si $f_{x,y}=f_{x}\times f_{y}$ (c'est-à-dire si la distribution de $X$ et $Y$ en paire est égale à la distribution de $X$ multipliée par la distribution de $Y$). Ce critère est équivalent à $f_{x|y}=f_{x}$ et $f_{y|x}=f_{y}$ (c'est-à-dire si la distribution de $X$ dans l'ensemble de la population est la même que la distribution de $X$ dans la sous-population définie par des valeurs spécifiques de $Y$).

Par exemple, supposons que nous voulions voir si une personne votant lors d'une élection ($X$) est indépendante de son sexe à la naissance ($Y$). Si ces variables étaient indépendantes, nous nous attendrions à ce que le pourcentage de femmes dans la population totale soit similaire au pourcentage de femmes parmi les personnes qui votent à l'élection. Cela correspond à notre définition de l'indépendance en statistiques.

### 3.1.2) Test du Chi-deux pour l'indépendance

Pour motiver le concept de test d'indépendance, considérons l'ensemble de données AOSI. Voyons si le site d'étude et le sexe sont indépendants. Rappelons que le tri à plat pour ces variables dans les données était le suivant.


In [65]:
table_exp_xtabs

,PHI,SEA,STL,UNC,Sum
Female,55,67,60,53,235
Male,94,85,85,88,352
Sum,149,152,145,141,587


In [66]:
prop_table_exp_xtabs

,PHI,SEA,STL,UNC,Sum
Female,0.09,0.11,0.10,0.09,0.39
Male,0.16,0.14,0.14,0.15,0.59
Sum,0.25,0.25,0.24,0.24,0.98


D'après notre définition de l'indépendance, il semble que le sexe et le site sont indépendants en comparant les effectifs dans chaque groupe de sexe et de site ainsi que les effectifs au niveau de la population. Effectuons un test formel pour voir s'il existe des preuves d'indépendance.

Tout d'abord, nous couvrons le **test du Chi-deux.** Pour tous ces tests, l'hypothèse nulle est que les variables sont indépendantes.

On calcule la valeur expectée pour chaque cellule du tableau, en utilisant la formule suivante :
$$
E_{i,j} = \frac{R_i C_j}{T}
$$

où :

* $E_{i,j}$ **:** La valeur expectée dans la cellule $(i,j).$
* $R_i$ **:** La somme de la ligne $i.$
* $C_j$ **:** La somme de la colonne $j.$
* $T$ **:** La somme totale du tableau

Sous cette hypothèse nulle, nous nous attendrions au tri à plat suivant :


In [69]:
# Construire un trableau de la même dimension que 'table_exp_xtabs'
expected_table <- table_exp_xtabs

# Peupler le tableau
sex_sums <- expected_table[,5]
site_sums <- expected_table[3,]
expected_table[1,1] <- 149*(235/587)
expected_table[2,1] <- 149*(352/587)

expected_table[1,2] <- 152*(235/587)
expected_table[2,2] <- 152*(352/587)

expected_table[1,3] <- 145*(235/587)
expected_table[2,3] <- 145*(352/587)

expected_table[1,4] <- 141*(235/587)
expected_table[2,4] <- 141*(352/587)

# Arrondissment
expected_table <- round(expected_table,2)

D'où viennent ces valeurs ? Prenons la colonne du site d'étude de Philadelphie comme exemple (étiquetée PHI). 

Comme expliqué précédemment, selon l'indépendance, à Philadelphie, nous nous attendons à ce que le pourcentage de participantes féminines soit le même que le pourcentage dans l'échantillon total. Il y a 149 participants de Philadelphie, 235 femmes et 587 sujets au total dans l'échantillon. L'échantillon total est d'environ 40% de femmes, donc nous nous attendrions à ce qu'il y ait environ 0.40*149, soit environ 59,6 femmes du site de Philadelphie et donc environ 89,4 hommes.

Autrement dit, l'effectif attendu est égal à (total de la ligne*total de la colonne)/taille de l'échantillon. Toutes les entrées sont calculées en utilisant cette équation. Regardons les différences entre les effectifs des données AOSI et les effectifs attendus.


In [70]:
expected_table

,PHI,SEA,STL,UNC,Sum
Female,59.65,60.85,58.05,56.45,235
Male,89.35,91.15,86.95,84.55,352
Sum,149.00,152.00,145.00,141.00,587


In [72]:
# On élimine la ligne et colonne qui reporésentent la somme
# (Expected - Observed) :
expected_table[-3,-5]-table_exp_xtabs[-3,-5]

        Study_Site
Gender     PHI   SEA   STL   UNC
  Female  4.65 -6.15 -1.95  3.45
  Male   -4.65  6.15  1.95 -3.45

Nous pouvons voir que les différences sont petites compte tenu des marges du site d'étude, donc il n'y a pas de preuves suggérant une dépendance.

Cependant, faisons cela de manière plus rigoureuse en utilisant un test d'hypothèse formel. Pour tout test d'hypothèse, nous créons une statistique de test, puis calculons une valeur $p$ à partir de cette statistique de test.

Informellement, la valeur $p$ **mesure la probabilité que nous observions une valeur de statistique de test aussi extrême ou plus extrême que la valeur observée dans l'ensemble de données si l'hypothèse nulle est vraie.**

Pour le **test du Chi-deux,** la statistique de test est égale à la somme des carrés des différences entre les effectifs observés et attendus, divisée par les effectifs attendus :

$$
\chi^2 = \sum \frac{(\text{Effectifs observés}-\text{Effectifs attendus})^2}{\text{Effectifs attendus}}
$$

La distribution de cette statistique de test est approximativement Chi-deux avec $\frac{r-1}{c-1}$ degrés de liberté, où :

* $r$ est le nombre de catégories de lignes.

* $c$ est le nombre de catégories de colonnes.

L'approximation devient plus précise à mesure que la taille de l'échantillon devient de plus en plus grande (jusqu'à l'infini). Ainsi, si la taille de l'échantillon est "suffisamment grande", nous pouvons approximer avec précision la distribution des statistiques de test avec cette distribution Chi-deux. C'est ce qui est appelé des statistiques à "grande échantillon" ou "asymptotique".

Réalisons le test du Chi-deux sur l'ensemble de données AOSI. Cela se fait en utilisant **`summary()`** avec l'objet de tri à plat (créé par `table()` ou `xtab()`). Notez que nous ne pouvons pas avoir les marges de lignes et de colonnes dans l'objet de tableau lors de la réalisation de ce test du Chi-deux, car R considérera que ces marges sont des catégories de lignes et de colonnes.


In [73]:
table_exp_xtabs <- xtabs(~Gender+Study_Site, data = data_aosi)
summary(table_exp_xtabs)

Call: xtabs(formula = ~Gender + Study_Site, data = data_aosi)
Number of cases in table: 587 
Number of factors: 2 
Test for independence of all factors:
	Chisq = 2.1011, df = 3, p-value = 0.5517

Nous voyons que la valeur $p$ est de $0,55,$ ce qui est très élevé et, en dessous d'un seuil de $0,05,$ est loin de la signification. Ainsi, nous n'avons pas de preuve pour rejeter l'hypothèse nulle selon laquelle le sexe et le site d'étude sont indépendants.


### 3.1.3) Test exact de Fisher

Une alternative au test du Chi-Square est le **test exact de Fisher.** Ce test d'hypothèse a les mêmes hypothèses nulle et alternative que le test du Chi-Square. Cependant, sa statistique de test a une distribution connue pour toute taille d'échantillon finie. Ainsi, aucune approximation de distribution n'est requise, contrairement au test du Chi-Square, et il produit donc des valeurs $p$ précises pour toute taille d'échantillon.

Pour effectuer le test exact de Fisher, on utilise la fonction **`fisher.test()`** du package `stats` avec l'objet `table` ou `xtab`.

L'inconvénient du test exact de Fisher est qu'il nécessite beaucoup de temps de calcul si les données ont une grande taille d'échantillon, dans ce cas, l'approximation du Chi-Square est probablement précise et cette procédure de test devrait être utilisée.

Exécutons le test exact de Fisher sur le tri à plat sexe par site :


In [75]:
?fisher.test

In [74]:
fisher.test(table_exp_xtabs)


	Fisher's Exact Test for Count Data

data:  table_exp_xtabs
p-value = 0.553
alternative hypothesis: two.sided


En raison de la taille de l'échantillon, nous constatons que la valeur $p$ est très proche de la valeur $p$ du test du Chi carré, comme prévu.


**Liens importants :**

* [Statistique bivariée](https://larmarange.github.io/analyse-R/statistique-bivariee.html)

* Pour plus d'approfondissement : [How to Make Beautiful Tables in R](https://rfortherestofus.com/2019/11/how-to-make-beautiful-tables-in-r)

# Change Log

| Date (YYYY-MM-DD) | Version | Changé Par      | Description du Changement |
| ----------------- | ------- | -------------   | ----------------------- |
| 2024-03-08       | 1.0     | Younes Dahami   |  version initiale|